<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/flax_stable_diffusion_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 亜麻の安定拡散動画

このノートブックでは、[Stable Diffusion](https://github.com/CompVis/stable-diffusion)の潜在空間を、推論を高速化するTPUを使って補間することで動画を生成することができます。

標準的なColab GPUと比較すると、初回実行後に〜6倍高速に動作します。最初の実行は、コードをコンパイルするので、GPU版と同等です。

...問題や機能に関する要望はこちらで受け付けています](https://github.com/nateraw/stable-diffusion-videos/issues)

お楽しみに🤗。

## Setup

In [ ]:
#@title JAXのセットアップ
#@markdown エラーが表示される場合は、TPUバックエンドを使用していることを確認してください。上のメニューで`Runtime`を選択し、"Change runtime type "というオプションを選択し、`Hardware accelerator`の設定で`TPU`を選択してください。
!pip install --upgrade jax jaxlib 

import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu('tpu_driver_20221011')

!pip install flax diffusers transformers ftfy
jax.devices()

In [ ]:
%%capture
! pip install stable_diffusion_videos

## アプリの実行 🚀」。

### インターフェイスの読み込み

このステップは、初めて実行する場合は数分かかります。

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp

from jax import pmap
from flax.jax_utils import replicate
from flax.training.common_utils import shard
from PIL import Image

from stable_diffusion_videos import FlaxStableDiffusionWalkPipeline, Interface

pipeline, params = FlaxStableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", 
    revision="bf16", 
    dtype=jnp.bfloat16
)
p_params = replicate(params)

interface = Interface(pipeline, params=p_params)

In [ ]:
#@title Google Driveと連携して出力を保存

#@markdown Google Driveを接続する場合は、下のチェックボックスをクリックし、このセルを実行してください。認証のためのプロンプトが表示されます。

#@markdown このColabセッションで出力したものを保存するだけなら、このセルを気にする必要はないでしょう

connect_google_drive = True #@param {type:"boolean"}

#@markdown そして、インターフェイスの「ビデオ」タブで、このパスを「出力」として使い、Google Driveに動画を保存します：

#@markdown > /content/gdrive/MyDrive/stable_diffusion_videos


if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

### 起動

Gradioのインターフェイスを起動するセルです。ここでは、その使い方を提案します：

1.画像」タブを使って、好きな画像を生成する - 間でモーフィングしたい2つの画像を見つける - これらの画像は同じ設定（ガイダンススケール、高さ、幅）を使用する - 使用した種/設定を記録しておくと、以下の ...<id-number>.gradio.app` 、フルスクリーンで見ることができるようになるはずです。

In [ ]:
interface.launch(debug=True)

---

## プログラムで `walk` を使用する

もう一つの選択肢は、インターフェイスを使用せず、代わりにプログラムで `walk` を使用することです。その方法は以下の通りです...

まず、Colabで動画を可視化するためのヘルパーfnを定義します。

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

Walk! 🚶‍♀️

In [ ]:
video_path = pipeline.walk(
    p_params,
    ['a cat', 'a dog'],
    [42, 1337],
    fps=5, # テストには5を、より良い品質を求めるなら25または30を使用する。
    num_interpolation_steps=30, # テストには3-5を使用し、より良い結果を得るには30以上を使用する。
    height=512, # 512を超える場合は64の倍数を使用。< 512の場合は8の倍数。
    width=512, # 512を超える場合は64の倍数を使用。< 512の場合は8の倍数。
    jit=True # 全てのTPUコアを使用する場合
    )
visualize_video_colab(video_path)

### ボーナス音楽ビデオ

まず、`youtube-dl`をインストールする必要があります。

In [ ]:
%%capture
! pip install youtube-dl

次に、例の音楽ファイルをダウンロードすることができます。
ここでは、私のサウンドクラウドから1つダウンロードします：

In [ ]:
! youtube-dl -f bestaudio --extract-audio --audio-format mp3 --audio-quality 0 -o "music/thoughts.%(ext)s" https://soundcloud.com/nateraw/thoughts

In [ ]:
from IPython.display import Audio

Audio(filename='music/thoughts.mp3')

In [ ]:
#曲中の秒数
audio_offsets = [7, 9]
fps = 8

# 秒をフレームに変換する
num_interpolation_steps = [(b-a) * fps for a, b in zip(audio_offsets, audio_offsets[1:])]

video_path = pipeline.walk(
    p_params,
    prompts=['blueberry spaghetti', 'strawberry spaghetti'],
    seeds=[42, 1337],
    num_interpolation_steps=num_interpolation_steps,
    height=512,                           # 64の倍数を使用する
    width=512,                             # 64の倍数を使用する
    audio_filepath='music/thoughts.mp3',   # 独自のファイルを使用する
    audio_start_sec=audio_offsets[0],     # 提供された音声のうち2つ目を開始する
    fps=fps,                               # 定義したnum_interpolation_stepsに基づいて設定することが重要です。
    batch_size=2,                          # TPU-v2では、512x512で最大3個です。
    output_dir='./dreams',                 # 画像が保存される場所
    name=None,                             # 出力ディレクトリのサブディレクトリは、デフォルトでタイムスタンプになります。
    jit=True                               # すべてのTPUコアを使用するために
)
visualize_video_colab(video_path)